## 1. Calculate the gene expression score

In [ ]:
import os,gc
import scanpy as sc
import pandas as pd
import anndata as ad
import numpy as np

os.chdir("/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD")

adata = sc.read('/data/wuqinhua/phase_1r/AD/AD_NatureNeuro_ADNC.h5ad')
print("read_over")

adata1 = adata[adata.obs['phenotype'] == "Not AD"]
adata2 = adata[adata.obs['phenotype'] == "Low"]
adata3 = adata[adata.obs['phenotype'] == "Intermediate"]
adata4 = adata[adata.obs['phenotype'] == "High"]
print("split_over")

H_gene = pd.read_csv("./ensemble_gene_attributions_Not_AD.csv") 
H_top_200 = H_gene.sort_values(by='mean_attribution', ascending=False).head(200)['gene'].tolist()
H_bottom_200 = H_gene.sort_values(by='mean_attribution', ascending=True).head(200)['gene'].tolist()
sc.tl.score_genes(adata1, H_top_200, score_name='gene_ex_score_top')
sc.tl.score_genes(adata1, H_bottom_200, score_name='gene_ex_score_bottom')
adata1.obs.to_csv('./Analysis/Figure/gene_ex_scores_Not_AD_top_bottom_200.csv')
print("Phenotype Not_AD scores saved.")

M_gene = pd.read_csv("./ensemble_gene_attributions_Low.csv") 
M_top_200 = M_gene.sort_values(by='mean_attribution', ascending=False).head(200)['gene'].tolist()
M_bottom_200 = M_gene.sort_values(by='mean_attribution', ascending=True).head(200)['gene'].tolist()
sc.tl.score_genes(adata2, M_top_200, score_name='gene_ex_score_top')
sc.tl.score_genes(adata2, M_bottom_200, score_name='gene_ex_score_bottom')
adata2.obs.to_csv('./Analysis/Figure/gene_ex_scores_Low_top_bottom_200.csv')
print("Phenotype Low scores saved.")

S_gene = pd.read_csv("./ensemble_gene_attributions_Intermediate.csv") 
S_top_200 = S_gene.sort_values(by='mean_attribution', ascending=False).head(200)['gene'].tolist()
S_bottom_200 = S_gene.sort_values(by='mean_attribution', ascending=True).head(200)['gene'].tolist()
sc.tl.score_genes(adata3, S_top_200, score_name='gene_ex_score_top')
sc.tl.score_genes(adata3, S_bottom_200, score_name='gene_ex_score_bottom')
adata3.obs.to_csv('./Analysis/Figure/gene_ex_scores_Intermediate_top_bottom_200.csv')
print("Phenotype Intermediate scores saved.")

Q_gene = pd.read_csv("./ensemble_gene_attributions_High.csv") 
Q_top_200 = Q_gene.sort_values(by='mean_attribution', ascending=False).head(200)['gene'].tolist()
Q_bottom_200 = Q_gene.sort_values(by='mean_attribution', ascending=True).head(200)['gene'].tolist()
sc.tl.score_genes(adata4, Q_top_200, score_name='gene_ex_score_top')
sc.tl.score_genes(adata4, Q_bottom_200, score_name='gene_ex_score_bottom')

# Save .obs, which now contains BOTH top and bottom scores
adata4.obs.to_csv('./Analysis/Figure/gene_ex_scores_High_top_bottom_200.csv')
print("Phenotype High scores saved.")

print("\nAll processing complete.")

In [ ]:
library(dplyr)
library(tidyr)
library(reshape2)
library(ggplot2)
library(ggrepel)
library(scales)
setwd("/home/wuqinhua/Project/PHASE_1r/AttnMoE_test/result/AD")

attnData <- read.csv('./AD_metadata.csv')
sampleInfo = read.csv('/data/wuqinhua/phase_1r/AD/sample_info.csv')

nameAll = unique(attnData$Supertype)
nameAll = sort(nameAll)
nameAll

nameList = nameAll

sampleFold = data.frame(id = character(), celltype = character(), fold = numeric())
idList = unique(attnData$sample_id)
for (id in idList) {
  attnTmp = attnData %>% filter(sample_id == id)
  avgScore = 1 / dim(attnTmp)[1]
  foldRes = attnTmp %>% group_by(Supertype) %>% summarise(res = median(log2(attention_weight_mean/avgScore)))
  dataTmp = data.frame(id = rep(id,dim(foldRes)[1]),
                       celltype = foldRes$Supertype,
                       fold = foldRes$res)
  dataTmp_s = dataTmp %>% filter(celltype %in% nameList)
  dataTmp_s$fold = scale(dataTmp_s$fold)
  sampleFold = rbind(sampleFold,dataTmp_s)
}

sampleFold.Table <- dcast(sampleFold, id ~ celltype)
rownames(sampleFold.Table) <- sampleFold.Table$id
sampleFold.Table$id <- NULL
attn <- sampleFold.Table

ex_H <- read.csv('./Analysis/Figure/gene_ex_scores_Not_AD_top_bottom_200.csv')
ex_M <- read.csv('./Analysis/Figure/gene_ex_scores_Low_top_bottom_200.csv')
ex_S <- read.csv("./Analysis/Figure/gene_ex_scores_Intermediate_top_bottom_200.csv")
ex_Q <- read.csv("./Analysis/Figure/gene_ex_scores_High_top_bottom_200.csv")

exData <- rbind(ex_H, ex_M, ex_S, ex_Q)

process_expression_data <- function(exData, sample_id_col, gene_ex_score_col) {
  nameAll <- unique(exData$Supertype)
  nameList <- sort(nameAll)
  
  sampleFold <- data.frame(id = character(), celltype = character(), fold = numeric())
  idList <- unique(exData[[sample_id_col]])
  
  for (id in idList) {
    attnTmp <- exData %>% filter(.data[[sample_id_col]] == id)
    foldRes <- attnTmp %>% group_by(Supertype) %>% summarise(res = median(.data[[gene_ex_score_col]]))
    dataTmp <- data.frame(id = rep(id, dim(foldRes)[1]),
                         celltype = foldRes$Supertype,
                         fold = foldRes$res)
    dataTmp_s <- dataTmp %>% filter(celltype %in% nameList)
    sampleFold <- rbind(sampleFold, dataTmp_s)
  }
  
  ex_sampleFold.Table <- dcast(sampleFold, id ~ celltype)
  rownames(ex_sampleFold.Table) <- ex_sampleFold.Table$id
  ex_sampleFold.Table$id <- NULL
  return(ex_sampleFold.Table)
}

ex_top <- process_expression_data(exData, 'sample_id', 'gene_ex_score_top')
ex_bottom <- process_expression_data(exData, 'sample_id', 'gene_ex_score_bottom')
sampleInfo <- sampleInfo[rownames(attn), ]
ex_top <- ex_top[rownames(attn), colnames(attn)]
ex_bottom <- ex_bottom[rownames(attn), colnames(attn)]

ex_diff <- ex_top - ex_bottom

custom_colors <- c("Not AD" = "#A2D8A2", "Low" = "#B8D8F1", "Intermediate" = "#F5B54F", "High" = "#d8b7ef")
sampleInfo = read.csv('/data/wuqinhua/phase_1r/AD/sample_info.csv')

attn_long <- attn %>%
  tibble::rownames_to_column("sample") %>%
  pivot_longer(cols = -sample, names_to = "cell_type", values_to = "attn_value")

ex_diff_long <- ex_diff %>%
  tibble::rownames_to_column("sample") %>%
  pivot_longer(cols = -sample, names_to = "cell_type", values_to = "ex_diff_value")

plot_data_long <- attn_long %>%
  left_join(ex_diff_long, by = c("sample", "cell_type")) %>%
  left_join(sampleInfo %>% select(sample_id,phenotype), by = c("sample" = "sample_id")) %>%
  mutate(group = factor(phenotype, levels = names(custom_colors))) %>%
  na.omit()


In [ ]:
annotation_data <- plot_data_long %>%
  group_by(cell_type) %>%
  summarise(
    cor_test_res = list(cor.test(attn_value, ex_diff_value, method = "pearson")),
    pcc_value = cor_test_res[[1]]$estimate,
    p_value = cor_test_res[[1]]$p.value
  ) %>%
  mutate(
    significance = case_when(
      p_value < 0.001 ~ "***",
      p_value < 0.01  ~ "**",
      p_value < 0.05  ~ "*",
      TRUE ~ ""
    ),
    title_with_sig = paste0(cell_type, " ", significance),
    label_string = paste0("PCC = ", round(pcc_value, 3), 
                          ", ", 
                          scales::pvalue(p_value, accuracy = 0.001, add_p = TRUE))
  )

plot_data_final <- plot_data_long %>%
  left_join(annotation_data %>% select(cell_type, title_with_sig), by = "cell_type")

final_plot <- ggplot(plot_data_final, aes(x = attn_value, y = ex_diff_value)) +
  geom_point(aes(color = group), alpha = 0.85, size = 2) +
  geom_smooth(
    method = "lm", se = TRUE,
    color = "#666666", fill = "#C9C9C9",
    alpha = 0.25, linewidth = 0.8
  ) +

  geom_text(
    data = annotation_data,
    aes(label = label_string),
    x = -Inf, y = Inf,     
    hjust = -0.1, vjust = 1.5, 
    size = 3
  ) +
  
  scale_color_manual(values = custom_colors, name = "Group") +
  
  facet_wrap(~title_with_sig, scales = "free",ncol = 10) +
  
  labs(
    title = "Attention vs. Expression Difference by Cell Type",
    x = "Standardized Attention Score",
    y = "Expression Score Difference (Top - Bottom)"
  ) +
  
  theme_classic(base_size = 12) +
  theme(
    panel.background = element_rect(fill = "white", color = NA),
    plot.background  = element_rect(fill = "white", color = NA),
    # panel.grid.major = element_line(color = "grey92", linewidth = 0.5),
    strip.background = element_rect(fill = "#EFEFEF", color = NA),
    strip.text = element_text(size = 10, face = "bold", color = "#333333"),
    axis.text = element_text(size = 9, color = "#333333"),
    axis.title = element_text(size = 11, face = "bold"),
    legend.position = "right",
    plot.title = element_text(hjust = 0.5, size = 16, face = "bold", margin = ggplot2::margin(b = 15)),
    panel.spacing = unit(1, "lines")
  )

ggsave("./Analysis/Figure/Con_plot/correlation_faceted_plot_Supertype_new.pdf",
       final_plot, width = 30, height = 40, dpi = 300)
print(final_plot)